# Adicionando um _Run_ Salvo ao MLFlow

In [ ]:
from utils.mlflow_run import log_run
run_path = '/bucket/models/PhilCNN/v1'
log_run(run_path)

## Falzendo alguns testes (Phil, vc pode ignorar tudo daqui pra baixo...)

In [ ]:
import mlflow
from utils.mlflow_run import MLFlow
import matplotlib.pyplot as plt
import seaborn as sns
import lpsds.notebook
from scripts.operation import ProductionModelOperation
from dorothy_sdk.dataset_wrapper import Dataset

In [ ]:
exp_name = 'Debug'
test_fold = 0
mlflow.set_experiment(exp_name)
mlflow.start_run()

run = MLFlow(run_path)
metrics = run.save_metrics()
test_sort = int(metrics.iloc[test_fold].sort)

In [ ]:
output = run.save_models_outputs(metrics)
output.drop(index=output.loc[output.fold != test_fold].index, inplace=True)
output.rename(columns={'y_true' : 'y_true_out'}, inplace=True)

In [ ]:
ds = Dataset('china').load_metadata()
ds = ds.merge(output, how='inner', on='image')
ds.rename(columns={'image_local_path' : 'path', 'y_true' : 'label'}, inplace=True)

In [ ]:
op = ProductionModelOperation('gpu')
X, _ = Dataset.load_from_file(ds, 'path', 'label')
X = op.prepare_images(X)

In [ ]:
model = run.load_model(test_fold, test_sort)
ds['y_proba_op'] = model.predict(X).flatten()

In [ ]:
plt.figure()
sns.histplot(data=ds, x='y_proba_op')

In [ ]:
plt.figure()
sns.scatterplot(data=ds, x='y_proba', y='y_proba_op')
plt.title('Scatterplot entre o Modelo Treinado e de Operação')
plt.xlabel('Modelo treinado (referência)')
plt.ylabel('Modelo de operação')
plt.grid()